In [3]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community faiss-cpu

In [25]:
# load test data
import pandas as pd

df = pd.read_csv('test_data/test_question_set_cleaned.csv')
df.head()

,Category,Question,Answer
0,HISTORY,"For the last 8 years of his life, Galileo was ...",Copernicus
1,ESPN's TOP 10 ALL-TIME ATHLETES,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,EVERYBODY TALKS ABOUT IT...,The city of Yuma in this state has a record av...,Arizona
3,THE COMPANY LINE,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,EPITAPHS & TRIBUTES,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [26]:
# Identify if there are any null cells
df.isnull().sum()

Category    0
Question    0
Answer      0
dtype: int64

In [13]:
# Convert test data to a format that can be used by the model - from CSV to SQLite
from pathlib import Path

# Load the data
Path('test_data.db').touch()


### Create SQL Database and Load Trivia Data Into It

In [ ]:
df = pandas.read_csv(test_data/test_question_set_cleaned.csv)
df.to_sql(test_data.db, conn, if_exists='append', index=False)

In [17]:
import csv, sqlite3

conn = sqlite3.connect('test_data.db')
cur = conn.cursor()

with open('test_data/test_question_set_cleaned.csv') as f:
    reader = csv.reader(f)
    data = list(reader)

In [21]:
cur.execute(''' 
CREATE TABLE test_question_set (
    category TEXT,
    question TEXT,
    answer TEXT);
''')

In [22]:
for row in data:
    cur.execute('INSERT INTO test_question_set VALUES (?, ?, ?)', row)

In [23]:
conn.commit()

In [24]:
conn.close()

In [ ]:
import csv, sqlite3

conn = sqlite3.connect('test_data.db')
cur = conn.cursor()
cur.execute("CREATE TABLE jeopardy_set (Category, Question, Answer)")

# Load the data into the Pandas DataFrame
with open ('test_data/test_question_set_cleaned.csv', 'r') as f:
    dr = csv.DictReader(f)
    to_db = [(i['Category'], i['Question'], i['Answer']) for i in dr]

cur.executemany("INSERT INTO t (Category, Question, Answer) VALUES (?, ?, ?);", to_db)
conn.commit()
conn.close()


In [11]:
# Test database connection

from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///test_data.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM jeopardy_set;")


sqlite
['jeopardy_set']


''

In [ ]:
# Enable tracing
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Call LLM Model

In [ ]:
# Add phi3:3.8b LLM model
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="phi3:3.8b",
)

In [ ]:
# Test LLM model is working
response_message = model.invoke("What is the capital of France?")

print(response_message.content)

In [ ]:
import pandas as pd
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage
import random

class QuizChatbot:
    def __init__(self, csv_path: str):
        # Load the CSV data
        self.df = pd.read_csv(csv_path)
        self.categories = self.df['Category'].unique().tolist()
        
        # Initialize the Ollama model
        self.llm = ChatOllama(
            model="phi3:3.8b",
            temperature=0
        )
        
        # Initialize state variables
        self.current_question = None
        self.current_answer = None
        
    def get_categories(self) -> str:
        """Return formatted list of categories"""
        return "\n".join([f"{i+1}. {cat}" for i, cat in enumerate(self.categories)])
    
    def select_question(self, category: str) -> tuple:
        """Select a random question from the specified category"""
        category_questions = self.df[self.df['Category'] == category]
        if len(category_questions) == 0:
            return None, None
        
        question_row = category_questions.sample(n=1).iloc[0]
        return question_row['Question'], question_row['Answer']
    
    def verify_answer(self, user_answer: str, correct_answer: str) -> bool:
        """Use the LLM to verify if the answer is correct"""
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are an answer verification expert. 
            Compare the user's answer with the correct answer and determine if they are semantically equivalent.
            Respond with only 'True' if correct or 'False' if incorrect."""),
            HumanMessage(content=f"""
            Correct answer: {correct_answer}
            User's answer: {user_answer}
            Are these answers semantically equivalent?
            """)
        ])
        
        response = self.llm.invoke(prompt.format_messages())
        return 'true' in response.content.lower()
    
    def run(self):
        """Main interaction loop"""
        print("Welcome to the Quiz Chatbot!")
        
        while True:
            print("\nAvailable categories:")
            print(self.get_categories())
            
            # Get category selection
            selection = input("\nPlease select a category number (or 'quit' to exit): ")
            
            if selection.lower() == 'quit':
                print("Thank you for playing!")
                break
                
            try:
                category_index = int(selection) - 1
                if 0 <= category_index < len(self.categories):
                    selected_category = self.categories[category_index]
                    
                    # Get random question from category
                    question, answer = self.select_question(selected_category)
                    if question is None:
                        print("Error: No questions available in this category.")
                        continue
                    
                    # Ask question and get user's answer
                    print(f"\nQuestion: {question}")
                    user_answer = input("Your answer: ")
                    
                    # Verify answer
                    is_correct = self.verify_answer(user_answer, answer)
                    
                    # Provide feedback
                    if is_correct:
                        print("Correct! Well done!")
                    else:
                        print(f"Incorrect. The correct answer was: {answer}")
                else:
                    print("Invalid category number. Please try again.")
            except ValueError:
                print("Please enter a valid number.")

# Usage
if __name__ == "__main__":
    chatbot = QuizChatbot("test_data/test_question_set_cleaned.csv")
    chatbot.run()


In [ ]:
import pandas as pd
from langchain.agents import Tool
from langchain.agents import AgentExecutor, CasualAgent
from langchain.memory import ConversationBufferMemory
import random

class QuizBot:
    def __init__(self, csv_path):
        # Load the CSV data
        self.df = pd.read_csv(csv_path)
        self.categories = self.df['category'].unique().tolist()
        self.current_question = None
        self.current_answer = None
        
        # Initialize LangChain components
        self.llm = ChatOllama(
            model="phi3:3.8b",
            temperature=0
        )
        self.memory = ConversationBufferMemory(memory_key="chat_history")
        
        # Define tools for the agent
        self.tools = [
            Tool(
                name="get_categories",
                func=self.get_categories,
                description="Get list of available categories"
            ),
            Tool(
                name="select_question",
                func=self.select_question,
                description="Select a random question from a category"
            ),
            Tool(
                name="verify_answer",
                func=self.verify_answer,
                description="Verify if the provided answer is correct"
            )
        ]
        
        # Create the agent
        self.agent = ConversationationalAgent.from_llm_and_tools(
            llm=self.llm,
            tools=self.tools,
            verbose=True
        )
        
        self.agent_executor = AgentExecutor.from_agent_and_tools(
            agent=self.agent,
            tools=self.tools,
            memory=self.memory,
            verbose=True
        )

    def get_categories(self):
        """Return available categories"""
        return f"Available categories: {', '.join(self.categories)}"

    def select_question(self, category):
        """Select a random question from the specified category"""
        category_questions = self.df[self.df['Category'] == category]
        if len(category_questions) == 0:
            return "Category not found"
        
        question_row = category_questions.sample(n=1).iloc[0]
        self.current_question = question_row['Question']
        self.current_answer = question_row['Answer']
        return self.current_question

    def verify_answer(self, user_answer):
        """Verify if the user's answer is correct"""
        if self.current_answer is None:
            return "No question has been selected yet"
        
        # Use the LLM to compare answers more flexibly
        prompt = f"""
        Compare these answers and determine if they are semantically equivalent:
        Correct answer: {self.current_answer}
        User answer: {user_answer}
        Respond with only 'True' if they match, 'False' if they don't.
        """
        
        response = self.llm.predict(prompt)
        is_correct = 'true' in response.lower()
        
        return "Correct!" if is_correct else f"Incorrect. The correct answer is: {self.current_answer}"

    def run(self):
        """Run the interactive quiz"""
        print("Welcome to the Quiz Bot!")
        print(self.get_categories())
        
        while True:
            user_input = input("\nPlease select a category or type 'quit' to exit: ")
            
            if user_input.lower() == 'quit':
                break
                
            if user_input in self.categories:
                question = self.select_question(user_input)
                print(f"\nQuestion: {question}")
                
                answer = input("Your answer: ")
                result = self.verify_answer(answer)
                print(result)
            else:
                print("Invalid category. Please choose from the available categories.")

# Usage
if __name__ == "__main__":
    quiz_bot = QuizBot("test_data/test_question_set_cleaned.csv")
    quiz_bot.run()
